## Report

We first split the features with three values into three features. I have used sklearn to perform classification using SVM as it internally uses LIBSVM. Kernel SVMs perform better than Linear SVM. Within Kernel SVMs, RBF kernel performed better than Polynomial kernel. 

The value of regularization parameter for Linear SVM which gave the best accuracy is 1.
The value of regularization parameter for Kernel SVM (both RBF and Polynomial) which gave the best accuracy is 10.

In [2]:
import numpy as np
import pandas as pd

In [3]:
names = []
for i in range(1,32):
    names.append(str(i))
data = pd.read_csv('hw2_question3.csv', names=names)

In [4]:
data.head()

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1


In [5]:
data['2'].unique()

array([ 1,  0, -1])

#### Performing One hot encoding. Converting features with 3 values into three new features

In [25]:
# One hot encoding data
# Handling categorical features
one_hot_encoded_data = pd.get_dummies(data, \
columns = ['2','7','8','14','15','16','26','29'])
one_hot_encoded_data.head()

,1,3,4,5,6,9,10,11,12,13,...,15_1,16_-1,16_0,16_1,26_-1,26_0,26_1,29_-1,29_0,29_1
0,-1,1,1,-1,-1,-1,1,1,-1,1,...,1,1,0,0,1,0,0,0,0,1
1,1,1,1,1,-1,-1,1,1,-1,1,...,0,1,0,0,0,1,0,0,0,1
2,1,1,1,1,-1,-1,1,1,-1,1,...,0,1,0,0,0,0,1,0,1,0
3,1,1,1,1,-1,1,1,1,-1,-1,...,0,1,0,0,0,0,1,1,0,0
4,1,-1,1,1,-1,-1,1,1,1,1,...,0,1,0,0,0,1,0,0,0,1


#### Train-test split

In [26]:
#train - test split

from sklearn.model_selection import train_test_split
train, test = train_test_split(one_hot_encoded_data,\
                               test_size=0.33)
print len(train), len(test)
train.head()

7406 3649


,1,3,4,5,6,9,10,11,12,13,...,15_1,16_-1,16_0,16_1,26_-1,26_0,26_1,29_-1,29_0,29_1
9007,1,1,1,1,-1,-1,1,1,1,1,...,1,1,0,0,1,0,0,0,1,0
9667,-1,1,-1,1,-1,1,-1,-1,1,1,...,0,1,0,0,1,0,0,0,1,0
217,1,1,1,1,-1,1,1,1,1,-1,...,0,1,0,0,0,0,1,0,0,1
107,1,1,1,1,-1,-1,1,1,1,-1,...,0,1,0,0,1,0,0,0,1,0
2714,1,1,1,1,-1,1,1,1,1,-1,...,0,1,0,0,0,1,0,0,0,1


In [19]:
## Separating Y data and X data


Y_train = train['31']
del train['31']
train['31'] = Y_train
X_train = train.loc[:,:"29_1"]
Y_test = test['31']
del test['31']
test['31'] = Y_test
X_test = test.loc[:,:"29_1"]

/Users/savinaynarendra/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/savinaynarendra/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Using libsvm with the help of sklearn

#### Linear SVM in LIBSVM. Regularization parameter has a value of [0.1, 1, 2, 5, 10]

In [27]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import time
X_train = np.array(X_train)
Y_train = np.array(Y_train)
kf = KFold(n_splits=3)
sum_time = 0
C = [0.1, 1, 2, 5, 10]
best_accuracy = 0
for c in C:
    accuracy = 0
    for train_index, test_index in kf.split(X_train):
        X_train_CV, X_test_CV = X_train[train_index],\
        X_train[test_index]
        Y_train_CV, Y_test_CV = Y_train[train_index],\
        Y_train[test_index]

        model = SVC(C=c, kernel='linear')
        tic = time.clock()
        model.fit(X_train_CV, Y_train_CV)
        toc = time.clock()
        sum_time += toc - tic
        Y_pred_CV = model.predict(X_test_CV)
        accuracy += accuracy_score(Y_test_CV, Y_pred_CV)
    accuracy = accuracy/3.0
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_c = c
avg_time = sum_time/15
print "Average training time: ", avg_time
print "Best value of Regularization parameter", best_c
print "Cross-Validation Accuracy for best C = ", best_c,\
"is: ", accuracy

Y_pred = best_model.predict(X_test)
accuracy_test = accuracy_score(Y_test, Y_pred)
print "Accuracy on Test Data with C = ", best_c, "is: ",\
accuracy_test

Average training time:  0.621835266667
Best value of Regularization parameter 1
Cross-Validation Accuracy for best C =  1 is:  0.938157846656
Accuracy on Test Data with C =  1 is:  0.934776651137


### Kernel SVM using RBF kernel

In [28]:
sum_time = 0
C = [0.1, 1, 2, 5, 10]
best_accuracy = 0
for c in C:
    accuracy = 0
    for train_index, test_index in kf.split(X_train):
        X_train_CV, X_test_CV = X_train[train_index], \
        X_train[test_index]
        Y_train_CV, Y_test_CV = Y_train[train_index], \
        Y_train[test_index]

        model = SVC(C=c, kernel='rbf')
        tic = time.clock()
        model.fit(X_train_CV, Y_train_CV)
        toc = time.clock()
        sum_time += toc - tic
        Y_pred_CV = model.predict(X_test_CV)
        accuracy += accuracy_score(Y_test_CV, Y_pred_CV)
    accuracy = accuracy/3.0
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_c = c
avg_time = sum_time/15
print "Average training time: ", avg_time
print "Best value of Regularization parameter", best_c
print "Cross-Validation Accuracy for best C = ", best_c, "is: ",\
accuracy

Y_pred = best_model.predict(X_test)
accuracy_test = accuracy_score(Y_test, Y_pred)
print "Accuracy on Test Data with C = ", best_c, "is: ",\
accuracy_test

Average training time:  0.446891533333
Best value of Regularization parameter 10
Cross-Validation Accuracy for best C =  10 is:  0.956386420135
Accuracy on Test Data with C =  10 is:  0.955056179775


### Kernel SVM using Polynomial Kernel

In [29]:
sum_time = 0
C = [0.1, 1, 2, 5, 10]
best_accuracy = 0
for c in C:
    accuracy = 0
    for train_index, test_index in kf.split(X_train):
        X_train_CV, X_test_CV = X_train[train_index],\
        X_train[test_index]
        Y_train_CV, Y_test_CV = Y_train[train_index],\
        Y_train[test_index]

        model = SVC(C=c, kernel='poly')
        tic = time.clock()
        model.fit(X_train_CV, Y_train_CV)
        toc = time.clock()
        sum_time += toc - tic
        Y_pred_CV = model.predict(X_test_CV)
        accuracy += accuracy_score(Y_test_CV, Y_pred_CV)
    accuracy = accuracy/3.0
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_c = c
avg_time = sum_time/15
print "Average training time: ", avg_time
print "Best value of Regularization parameter", best_c
print "Cross-Validation Accuracy for best C = ", best_c,\
"is: ", accuracy

Y_pred = best_model.predict(X_test)
accuracy_test = accuracy_score(Y_test, Y_pred)
print "Accuracy on Test Data with C = ", best_c, "is: ",\
accuracy_test

Average training time:  0.505529466667
Best value of Regularization parameter 10
Cross-Validation Accuracy for best C =  10 is:  0.956791661224
Accuracy on Test Data with C =  10 is:  0.954508084407


### Based on the results of Kernel SVM, I would prefer to use RBF kernel as the average training time is less for RBF kernel than polynomial kernel and the accuracy on test data on the best regularization parameter is almost same. We also observe that Kernel SVM performs better than Linear SVM.